In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import torch
import torch.nn as nn

In [ ]:
df_train = pd.read_csv('../input/new-york-city-taxi-fare-prediction/train.csv')

In [ ]:
df_train.head()

In [ ]:
df_train = df_train[:20000]

In [ ]:
df_train.shape

In [ ]:
def haversine_distance(df, lat1, long1, lat2, long2):
    r = 6371
    phi1 = np.radians(df[lat1])
    phi2 = np.radians(df[lat2])
    
    delta_phi = np.radians(df[lat2] - df[lat1])
    delta_lambda = np.radians(df[long2] - df[long1])
    
    a = np.sin(delta_phi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = (r * c)
    
    return d

In [ ]:
df_train['distance_kms'] = haversine_distance(df_train, 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude')

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train['pickup_datetime'] = pd.to_datetime(df_train['pickup_datetime'])

In [ ]:
# converting UTC to New York Timezone
df_train['NY_time'] = df_train['pickup_datetime'] - pd.Timedelta(hours=4)

In [ ]:
df_train['Hour']= df_train['NY_time'].dt.hour

In [ ]:
df_train['AMPM'] = df_train['Hour'].apply(lambda x: 'am' if x <= 12 else 'pm')

In [ ]:
df_train['Dayofweek'] = df_train['NY_time'].dt.dayofweek

In [ ]:
df_train['Month'] = df_train['NY_time'].dt.month

In [ ]:
df_train.head()

In [ ]:
cat_cols = ['Hour','AMPM','Dayofweek','Month']
cont_cols = ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'passenger_count', 'distance_kms']

In [ ]:
df_train.dtypes

In [ ]:
for cat in cat_cols:
    df_train[cat] = df_train[cat].astype('category')

In [ ]:
cats = np.stack([df_train[col].cat.codes.values for col in cat_cols], axis=1)

In [ ]:
conts = np.stack([df_train[col].values for col in cont_cols], axis=1)

In [ ]:
cats = torch.tensor(cats, dtype=torch.int64)
conts = torch.tensor(conts, dtype=torch.float)

In [ ]:
conts

In [ ]:
y = torch.tensor(df_train['fare_amount'].values, dtype = torch.float).reshape(-1,1)

In [ ]:
cat_size = [len(df_train[col].cat.categories) for col in cat_cols]

In [ ]:
cat_size

In [ ]:
embedding_size = [(size,min(50,(size+1)//2)) for size in cat_size]

In [ ]:
embedding_size

In [ ]:
class TabularModel(nn.Module):

    def __init__(self, embedding_size, n_cont, out_sz, layers, p=0.5):
        super().__init__()
        self.embeds = nn.ModuleList([nn.Embedding(ni, nf) for ni,nf in embedding_size])
        self.emb_drop = nn.Dropout(p)
        self.bn_cont = nn.BatchNorm1d(n_cont)
        
        layerlist = []
        n_emb = sum((nf for ni,nf in embedding_size))
        n_in = n_emb + n_cont
        
        for i in layers:
            layerlist.append(nn.Linear(n_in,i)) 
            layerlist.append(nn.ReLU(inplace=True))
            layerlist.append(nn.BatchNorm1d(i))
            layerlist.append(nn.Dropout(p))
            n_in = i
        layerlist.append(nn.Linear(layers[-1],out_sz))
            
        self.layers = nn.Sequential(*layerlist)
    
    def forward(self, x_cat, x_cont):
        embeddings = []
        for i,e in enumerate(self.embeds):
            embeddings.append(e(x_cat[:,i]))
        x = torch.cat(embeddings, 1)
        x = self.emb_drop(x)
        
        x_cont = self.bn_cont(x_cont)
        x = torch.cat([x, x_cont], 1)
        x = self.layers(x)
        return x

In [ ]:
torch.manual_seed(33)
model = TabularModel(embedding_size, conts.shape[1], 1, [200,100,50,20,10])

In [ ]:
model

In [ ]:
criterion = nn.MSELoss()  # we'll convert this to RMSE later
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
batch_size = len(df_train)
test_size = int(batch_size * .2)

cat_train = cats[:batch_size-test_size]
cat_test = cats[batch_size-test_size:batch_size]
con_train = conts[:batch_size-test_size]
con_test = conts[batch_size-test_size:batch_size]
y_train = y[:batch_size-test_size]
y_test = y[batch_size-test_size:batch_size]

In [ ]:
import time
start_time = time.time()

epochs = 300
losses = []

for i in range(epochs):
    i+=1
    y_pred = model(cat_train, con_train)
    loss = torch.sqrt(criterion(y_pred, y_train)) # RMSE
    losses.append(loss)
    
    # a neat trick to save screen space:
    if i%25 == 1:
        print(f'epoch: {i:3}  loss: {loss.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'epoch: {i:3}  loss: {loss.item():10.8f}') # print the last line
print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

In [ ]:
# TO EVALUATE THE ENTIRE TEST SET
with torch.no_grad():
    y_val = model(cat_test, con_test)
    loss = torch.sqrt(criterion(y_val, y_test))
print(f'RMSE: {loss:.8f}')

In [ ]:
print(f'{"PREDICTED":>13} {"ACTUAL":>7} {"DIFF":>7}')
for i in range(50):
    diff = np.abs(y_val[i].item()-y_test[i].item())
    print(f'{i+1:2}. {y_val[i].item():8.4f} {y_test[i].item():8.4f} {diff:8.4f}')

In [ ]:
# Make sure to save the model only after the training has happened!
if len(losses) == epochs:
    torch.save(model.state_dict(), 'TaxiFareRegrModel.pt')
else:
    print('Model has not been trained. Consider loading a trained model instead.')

In [ ]:
df_test = pd.read_csv('../input/new-york-city-taxi-fare-prediction/train.csv')

In [ ]:
df_test.head()

In [ ]:
df_test['distance_kms'] = haversine_distance(df_test, 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude')

In [ ]:
df_test['pickup_datetime'] = pd.to_datetime(df_test['pickup_datetime'])

In [ ]:
# converting UTC to New York Timezone
df_test['NY_time'] = df_test['pickup_datetime'] - pd.Timedelta(hours=4)

In [ ]:
df_test['Hour']= df_test['NY_time'].dt.hour

In [ ]:
df_test['AMPM'] = df_test['Hour'].apply(lambda x: 'am' if x <= 12 else 'pm')

In [ ]:
df_test['Dayofweek'] = df_test['NY_time'].dt.dayofweek

In [ ]:
df_test['Month'] = df_test['NY_time'].dt.month

In [ ]:
df_test.head()

In [ ]:
df_test.shape

In [ ]:
for cat in cat_cols:
    df_test[cat] = df_test[cat].astype('category')

In [ ]:
cats = np.stack([df_test[col].cat.codes.values for col in cat_cols], axis=1)

In [ ]:
conts = np.stack([df_test[col].values for col in cont_cols], axis=1)

In [ ]:
cats_test = torch.tensor(cats, dtype=torch.int64)
conts_test = torch.tensor(conts, dtype=torch.float)

In [ ]:
y_pred = model(cats_test, conts_test)

In [ ]:
y_pred

In [ ]:
df_test['fare_amount'] = y_pred.detach().numpy()

In [ ]:
df_test.head()

In [ ]:
df_submission = df_test[['key','fare_amount']]

In [ ]:
df_submission.head()